# Prompting and basic Langchain

Importing necessary libraries and installing required packages

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
import pprint
from typing import List

from IPython.display import display, Markdown

In [ ]:
# Load environment variables from .env file
load_dotenv()

In [ ]:
# %pip install langchain langchain-openai langchain-groq langchain-community langchain-chroma

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
# from langchain_openai import OpenAIEmbeddings  # ❌ Requiere OPENAI_API_KEY (de pago)
from langchain_huggingface import HuggingFaceEmbeddings  # ✅ Gratis, local
from langchain_chroma import Chroma
from pydantic import BaseModel, Field

## A simple LLM-based Chat

For Groq, you need to get first an account and API KEY at https://groq.com/ 

The API KEY should go to the env variables

In [ ]:
llm_model = os.environ["OPENAI_MODEL"]
# llm_model = "moonshotai/kimi-k2-instruct"
print(llm_model)

# llm = ChatOpenAI(model=llm_model, temperature=0.1)
llm = ChatGroq(model=llm_model, temperature=0.1)

response = llm.invoke("Tell me a joke about data scientists")
print(response.content)

Chat models are based on roles and messages. Let's do our first chain with Langchain

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm # Chaining the prompt and the llm call

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg.content)

This is a another typical kind of chain. Here, we use a *zero-shot* mode, as the LLM answers with its own (pretrained) knowledge.

In [ ]:
SYSTEM_PROMPT = """You are an experienced software architect that assists a novice developer 
to design a system. """

qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", SYSTEM_PROMPT),
            ("human", "{question}"),
        ]
)

qa_chain = qa_prompt | llm

query = "What are the pros and cons of the Proxy design pattern?"
result = qa_chain.invoke(input=query)
# print(result.content)

display(Markdown(result.content)) # Result in Markdown format


## Handling Memory (as part of a chat)

The memory is a list of previous (pairs of) messages between the human and the assistant, which provide *context* for the next iteraction.

In [ ]:
CONTEXTUALIZED_PROMPT = """Given a chat history and the latest developer's question
    which might reference context in the chat history, formulate a standalone question
    that can be understood without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return it as is."""

contextualized_qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", CONTEXTUALIZED_PROMPT),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    )

# This is a possible chain to keep (and compress) past interactions
# It's a form of rewriting
contextualized_qa_chain = contextualized_qa_prompt | llm | StrOutputParser()


# A buffer to store messages from user and assistant
chat_history = InMemoryChatMessageHistory()
chat_history.add_user_message(query)
chat_history.add_ai_message(result)


query = "Can I combine the pattern with other patterns?"
ai_msg = contextualized_qa_chain.invoke(
    {
        'question': query, 
        'chat_history': chat_history.messages
    }
)
print(ai_msg)

Let's use the (compressed) history to answer the new question

In [ ]:
def contextualized_question(input: dict):
        if input.get("chat_history"):
            return contextualized_qa_chain
        else:
            return input["question"]

# A new QA chain that reuses the previous chain
qa_chain_with_memory = (
         RunnablePassthrough.assign(
            context=contextualized_question | qa_prompt | llm
        )
    )

result = qa_chain_with_memory.invoke(
    {
        'question': query,  
        'chat_history': chat_history.messages
    }
)

display(Markdown(result['context'].content))

In [ ]:
# How the whole result looks like
pprint.pprint(result)

## Handling Few-shots

In [ ]:
zero_shot_prompt = PromptTemplate(
    input_variables=['input'],
    template="""Return the antonym of the input given along with an explanation.
    Input: {input}
    Output:
    Explanation:
    """
)

# Zero-shot chain
zero_shot_chain = zero_shot_prompt | llm

query = 'I am very sad but still have hope'
result = zero_shot_chain.invoke(input=query)
print(result.content)

In [ ]:
# Examples of a task for creating antonyms.
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    # OpenAIEmbeddings(),  # ❌ Antiguo: requiere OPENAI_API_KEY
    HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),  # ✅ Nuevo: gratis, local
    Chroma, # The database to store the examples with their embeddings
    # The number of examples to produce.
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Return the antonym of the given input along with an explanation. \n\nExample(s):",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

print(similar_prompt.format(adjective="rainy"))


And let's use the new prompt in a chain

In [ ]:
# Few-shot
few_shot_chain = similar_prompt | llm

query = 'rainy' # 'I am very sad but still have hope'
print(similar_prompt.format(adjective=query))
print()

result = few_shot_chain.invoke(input=query)
print(result.content)

We can ask an LLM to generate its response as a JSON object, using the Pydantic framework. 

For example, we can format the antonym output

In [ ]:
class FormattedAntonym(BaseModel):
    antonym: str = Field(description="An antonym for the input word or phrase.")
    explanation: str = Field(description="A short explanation of how the antonym was generated")
    additional_clarifications: List[str] = Field(description="A list of questions that the LLM needs to clarify in order to ...", default=[])


llm_with_structure = llm.with_structured_output(FormattedAntonym)

few_shot_chain1 = similar_prompt | llm_with_structure
result = few_shot_chain1.invoke(input=query)
result # The Pydantic (object) format

In [ ]:
# print(result.model_dump_json(indent=2))
pprint.pprint(result.model_dump()) # This is a dict

---